##Assignment14
###group effort : Yadunandan,Deepak, Kayal, Manjunath H , abhigoku10, Parinita R9



### Import the required libraries





In [0]:

import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

import tensorflow.keras.backend as K
from tensorflow.keras import initializers
from tensorflow.keras.initializers import Initializer
from tensorflow.keras import regularizers
from tensorflow.keras import constraints
from tensorflow.keras.layers import InputSpec
from keras.backend.tensorflow_backend import _regular_normalize_batch_in_training

from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img
import numpy as np
import os
from keras.utils import np_utils
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import glob
from PIL import Image
#import tf.contrib.mixed_precision.LossScaleOptimizer


Using TensorFlow backend.


### tfrecords for  train and test is already available from preprocessing

In [0]:
!ls -la /output
    

total 455928
-rw-r--r-- 1 root root     51743 Aug 26 01:40  final_above95_colab_190sec_3Eoch_targetlr_with_cut_out_1024_RohanDN.ipynb
drwxr-xr-x 4 root root      4096 Aug 26 01:40 .
drwxr-xr-x 1 root root      4096 Aug 26 00:32 ..
-rw-r--r-- 1 root root       129 Aug 26 00:32 .floydignore
drwxr-xr-x 2 root root      4096 Aug 26 00:34 .ipynb_checkpoints
drwxr-xr-x 3 root root      4096 Aug 26 00:50 .keras
-rw-r--r-- 1 root root    217610 Aug 26 00:32 Assignment_14_DN_with_cutout_93_1-2_floyd_1.9_run.ipynb
-rw-r--r-- 1 root root    198810 Aug 26 00:32 DN_with_cutout_93%.ipynb
-rw-r--r-- 1 root root    204072 Aug 26 00:32 DN_with_cutout_93_1.ipynb
-rw-r--r-- 1 root root       173 Aug 26 01:34 MODELACCY1.txt
-rw-r--r-- 1 root root      2577 Aug 26 00:32 Untitled.rtf
-rw-r--r-- 1 root root    252576 Aug 26 00:32 Updated_run_re_floydgroup_Init16_08_RohanDN.ipynb
-rw-r--r-- 1 root root     12197 Aug 26 00:32 epoch_output_cloud.rtf
-rw-r--r-- 1 root root       479 Aug 26 00:32 floyd.yml
-rw-r-

In [0]:
tf.enable_eager_execution()
K.set_floatx('float16')
K.set_epsilon(1e-4) #default is 1e-7

In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = .4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 3 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float16, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random_uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

### Model building blocks

Create Convolution -> Batch Norm -> Activation layers

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out , bnorm = True):
    super().__init__()
    self.bnorm = bnorm
    #self.depth = tf.keras.layers.DepthwiseConv2D(kernel_size=3, padding="SAME", activation = "relu" , depthwise_regularizer =tf.initializers.glorot_normal() , use_bias=False)
    #self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=1, padding="SAME", kernel_initializer=tf.initializers.glorot_normal(), use_bias=False)
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=tf.keras.initializers.glorot_normal(), use_bias=False)
    if self.bnorm:
      self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    #print("con",inputs.shape)
    h = self.conv(inputs)
    if self.bnorm:
      h = self.bn(h) 
    return tf.nn.relu(h)
    #return tf.nn.relu(self.bn(((self.conv((inputs)))))) #self.bn

#### Create Residual block 

main line - ConvBN -> ConvBN -> Maxpool

residual line - ConvBN -> ConvBN -> ConvBN

concatenate the result of main line and residual line

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.conv_bn2 = ConvBN(c_out)
    
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)
      self.res3 = ConvBN(c_out)



  def call(self, inputs):
    h = self.pool(self.conv_bn2(self.conv_bn(inputs))) #run3 add extra conv roll back
    if self.res:
      #h = h + self.res2(self.res1(h)) # prev
      h = tf.keras.layers.concatenate([h,self.res3(self.res2(self.res1(h)))])
    return h

#### Build the model

ConvBN -> ConvBN -> Residual block 1 -> Residual block 2 -> Global Avg Pool -> softmax

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.sp_drpt = tf.keras.layers.SpatialDropout2D(0.2)
    self.init_conv_bn = ConvBN(c)
    self.conv = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=tf.keras.initializers.glorot_normal(), use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool((self.blk3((((self.blk1((self.init_conv_bn(x))))))))) # run2 add conv roll back self.blk2 self.sp_drpt
    h = self.linear(h) * self.weight
    #print("h ", h.dtype , " : lab : ",y.dtype)
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(h, axis = 1), y), tf.float32)) # math is removed PBO
    return loss, correct

In [0]:
"""from google.colab import drive
drive.mount('/content/gdrive')

"""#/content/gdrive/My Drive/Colab Notebooks/eva/dataset/cifar10

### Load the dataset

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import numpy as np

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.datasets.cifar import load_batch
from tensorflow.python.keras.utils.data_utils import get_file
#from tensorflow.python.util.tf_export import keras_export


# @keras_export('keras.datasets.cifar10.load_data')
def load_data():
  """Loads CIFAR10 dataset.
  Returns:
      Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
  """
  dirname = 'cifar-10-batches-py'
  origin = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
  path = get_file(dirname, origin=origin, untar=True)
#   path = dirname #fix
  num_train_samples = 50000

  x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
  y_train = np.empty((num_train_samples,), dtype='uint8')

  for i in range(1, 6):
    fpath = os.path.join(path, 'data_batch_' + str(i))
    (x_train[(i - 1) * 10000:i * 10000, :, :, :],
     y_train[(i - 1) * 10000:i * 10000]) = load_batch(fpath)

  fpath = os.path.join(path, 'test_batch')
  x_test, y_test = load_batch(fpath)

  y_train = np.reshape(y_train, (len(y_train), 1))
  y_test = np.reshape(y_test, (len(y_test), 1))

  if K.image_data_format() == 'channels_last':
    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)

  return (x_train, y_train), (x_test, y_test)

In [0]:
(x_train, y_train), (x_test, y_test) = load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

#x_train = normalize(
x_train = pad4(x_train)
x_test = normalize(pad4(x_test))  ##PBO pad4 added

In [0]:
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((50000, 40, 40, 3), (50000,)), ((10000, 40, 40, 3), (10000,)))

In [0]:
num_train, img_channels, img_rows, img_cols =  x_train.shape
# save the test dataset statistics
num_test, _, _, _ =  x_test.shape
# save the number of unique classes in the train dataset
num_classes = len(np.unique(y_train))

#### Initialize the dimension parameters

In [0]:
IMAGE_HEIGHT = 32 + 8
IMAGE_WIDTH = 32 + 8
IMAGE_DEPTH = 3
NUM_CLASSES = 10

#### External code used to cutout the image

In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

#### Create the imagedatagenerator for train and test data

In [0]:
from keras.preprocessing.image import ImageDataGenerator



# Image augmentation is done to increase the robustness of the model
datagen = ImageDataGenerator(zoom_range=0.0,
                             rotation_range=30 ,
                             featurewise_center=True,
                             featurewise_std_normalization=True,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(v_l=0, v_h=1))

test_datagen = ImageDataGenerator(zoom_range=0.0,
                             featurewise_center=False,
                             featurewise_std_normalization=False,
                             horizontal_flip=False)

#### This code is used to create a folder to store the augmented images
dir_augmented_data = "/root/.keras/augment"
try:
  ##### if the preview folder does not exist, create
  os.mkdir(dir_augmented_data)
  
except:
  ##### if the preview folder exists, then remove
  ##### the contents (pictures) in the folder
  for item in os.listdir(dir_augmented_data):
    os.remove(dir_augmented_data + "/" + item)

In [0]:
#!ls /root/.keras/augment

#### This code is used to augment the train image and save to directory

##### Segregate the train data based on the class 
##### Then for each record create 4 augmented images and save to directory
##### each file is prefixed with the class id , this information is used to create the label array for the images later

class_ids = [0,1,2,3,4,5,6,7,8,9]

for j in range(num_classes):
  idx = np.where(y_train[:]==j)[0]
  features_idx = x_train[idx,::]
  print(features_idx.shape)
  
  for i in range(len(features_idx)):
    
    i = 0
    Nplot = 4
    x = img_to_array(features_idx[i])
    ## reshape (Sampke, Nrow, Ncol, 3) 3 = R, G or B
    x = x.reshape((1,) + x.shape)
    for batch in datagen.flow(x ,batch_size=32,
                          save_to_dir=dir_augmented_data,
                          save_prefix= class_ids[j], # "0",
                          save_format='png'):
      i += 1
      
      if i > Nplot - 1: ## generate 8 pictures 
        break    


In [0]:
!ls -la    /output/.keras/augment/tfrecord


total 45136
drwxr-xr-x 2 root root     4096 Aug 26 01:33 .
drwxr-xr-x 3 root root     4096 Aug 26 00:51 ..
-rw-r--r-- 1 root root 46210080 Aug 26 01:33 test.tfrecords-000


In [0]:
#!du -h /root/.keras/augment

In [0]:
#!rm -rf /output/.keras/augment/tfrecord/

##### 2 folders are created to save the augmented image and tfrecord

In [0]:
!mkdir /output/.keras/augment/
!mkdir /output/.keras/augment/tfrecord/

mkdir: cannot create directory ‘/output/.keras/augment/’: File exists
mkdir: cannot create directory ‘/output/.keras/augment/tfrecord/’: File exists


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!cp  "/content/drive/My Drive/Colab Notebooks/eva/Session14/tfrecord/test.tfrecords-000" /root/.keras/augment/tfrecord/
#!cp  "/content/drive/My Drive/Colab Notebooks/eva/Session14/tfrecord/train.tfrecords-000" /root/.keras/augment/tfrecord/

##### If the tfrecord is created then the same is copied to a local folder

In [0]:
!cp  "/output/test.tfrecords-000" /output/.keras/augment/tfrecord/
!cp  "/output/test.tfrecords-000" /root/.keras/augment/tfrecord/
#!cp  "/content/drive/My Drive/Colab Notebooks/eva/Session14/tfrecord/train.tfrecords-000" /root/.keras/augment/tfrecord//train.tfrecords-000" /root/.keras/augment/tfrecord/

cp: cannot create regular file '/root/.keras/augment/tfrecord/': No such file or directory


#### This code is used to create the tfrecord writer handle

tfrecord file path

TF_recordPath = "/output/.keras/augment/tfrecord/"
##### the best number of images stored in each tfrecord file
bestNum = 1000
##### the index of images flowing into each tfrecord file
num = 0
##### the index of the tfrecord file
recordFileNum = 0

"""
##### name  of the tfrecord files
recordFileName = ("train.tfrecords-%.3d" % recordFileNum)
recordFileName_test = ("test.tfrecords-%.3d" % recordFileNum)

##### tfrecord file writer
writer = tf.io.TFRecordWriter(TF_recordPath + recordFileName)

writer_test = tf.io.TFRecordWriter(TF_recordPath + recordFileName_test)

In [0]:
#  tfrecord file path

TF_recordPath = "/output/.keras/augment/tfrecord/"
# the best number of images stored in each tfrecord file
bestNum = 1000
# the index of images flowing into each tfrecord file
num = 0
# the index of the tfrecord file
recordFileNum = 0

recordFileName = ("train.tfrecords-%.3d" % recordFileNum)
recordFileName_test = ("test.tfrecords-%.3d" % recordFileNum)

"""
# name  of the tfrecord files

# tfrecord file writer
writer = tf.io.TFRecordWriter(TF_recordPath + recordFileName)



writer_test = tf.io.TFRecordWriter(TF_recordPath + recordFileName_test)"""

##### int64 and bytes wrapper

In [0]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [0]:
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[(value)]))

#### This code is used to create the tfrecord from the augmented images


output_file = TF_recordPath + recordFileName
with tf.python_io.TFRecordWriter(output_file) as record_writer:
  for infile in glob.glob("/root/.keras/augment/*.png"): 
    
    index1=infile.find('augment/') + 8
    TF_label = int(infile[index1:index1 +1])
    im = Image.open(infile)
    img_raw = im.tobytes() 
    example = tf.train.Example(features=tf.train.Features(
          feature={
            'image': _bytes_feature(img_raw),
            'label': _int64_feature(TF_label)
          }))     
    record_writer.write(example.SerializeToString())


#### This code is used to save the test images into a local folder

##### There is no augmentation done on the test data

dir_test_data='/root/.keras/test'
os.mkdir(dir_test_data)
for j in range(num_classes):
  idx = np.where(y_test[:]==j)[0]
  features_idx_test = x_test[idx,::]
  print(features_idx_test.shape)
  
  for i in range(len(features_idx_test)):
    i = 0
    Nplot = 1
    x = img_to_array(features_idx_test[i])
    ## reshape (Sampke, Nrow, Ncol, 3) 3 = R, G or B
    x = x.reshape((1,) + x.shape)
    for batch in test_datagen.flow(x ,batch_size=32,
                          save_to_dir=dir_test_data,
                          save_prefix= class_ids[j], # "0",
                          save_format='png'):
      i += 1
      
      if i > Nplot - 1: ## generate 8 pictures 
        break   

#### This code is used to create tfrecord for the test data

output_file_test = TF_recordPath + recordFileName_test
with tf.python_io.TFRecordWriter(output_file_test) as record_writer:
  for infile in glob.glob("/root/.keras/test/*.png"):
  
    index1=infile.find('test/') + 5
    TF_label = int(infile[index1:index1 +1])
    im = Image.open(infile)
    img_raw = im.tobytes() 
    example = tf.train.Example(features=tf.train.Features(
          feature={
            'image': _bytes_feature(img_raw),
            'label': _int64_feature(TF_label)
          }))     
    record_writer.write(example.SerializeToString())

"""print("Creating the single tfrecord file")  

for infile in glob.glob("/root/.keras/augment/*.png"):
  index1=infile.find('augment/') + 8
  TF_label = int(infile[index1:index1 +1])
  im = Image.open(infile)
  img_raw = im.tobytes()
  example = tf.train.Example(features=tf.train.Features(feature={
            "img_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[TF_label]))}))

  writer.write(example.SerializeToString())"""

#### Create file path for train and test data tfrecord

In [0]:
tfrecord_location = TF_recordPath
name = recordFileName
filename = os.path.join(tfrecord_location, name)

In [0]:
name_test = recordFileName_test
filename_test = os.path.join(tfrecord_location, name_test)

In [0]:
!ls /output/.keras/augment/tfrecord/

test.tfrecords-000


In [0]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [0]:
#!cp /root/.keras/augment/tfrecord/* "/content/drive/My Drive/Colab Notebooks/eva/Session14/tfrecord"

In [0]:
#!ls "/content/drive/My Drive/Colab Notebooks/eva/Session14/tfrecord"

#### Create the train and test dataset from the saved tfrecord

In [0]:
dataset = tf.data.TFRecordDataset(filename)
dataset_test = tf.data.TFRecordDataset(filename_test)

In [0]:
type(dataset)

tensorflow.python.data.ops.readers.TFRecordDataset

#### External source code used to parse each record and convert to image tensor and label tensor

In [0]:
def parse_record(serialized_example):
  features = tf.parse_single_example(
    serialized_example,
    features={
      'image': tf.FixedLenFeature([], tf.string),
      'label': tf.FixedLenFeature([], tf.int64),
    })
  
  image = tf.decode_raw(features['image'], tf.uint8)
  image.set_shape([IMAGE_DEPTH * IMAGE_HEIGHT * IMAGE_WIDTH])
  image = tf.reshape(image, [IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH])
  image = tf.cast(tf.transpose(image, [1, 2, 0]), tf.float32)
  image = tf.random_crop(image,[ IMAGE_HEIGHT - 8 , IMAGE_WIDTH - 8 , IMAGE_DEPTH ])
  
  label = tf.cast(features['label'], tf.int64)
  #label = tf.one_hot(label, NUM_CLASSES)

  return image, label

#### preprocessing ahead of time

##### shuffle the tran dataset
##### parse the train and test dataset
##### preset the batch size in both dataset
##### prefetch 1 batch of train dataset

In [0]:
buffer_size = BATCH_SIZE * 2 + 1
dataset = dataset.shuffle(buffer_size=buffer_size)

dataset = dataset.map(parse_record ) #, num_parallel_calls= 10)

#dataset_test = dataset_test.shuffle(buffer_size=buffer_size)
dataset_test = dataset_test.map(parse_record )

In [0]:
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(1)

dataset_test = dataset_test.batch(BATCH_SIZE)
dataset = dataset_test.prefetch(1)

#### Create the model and define the lr schedule

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

schedule = np.interp(np.arange(EPOCHS+1), 
                     [0, int((EPOCHS+1)*0.2), int((EPOCHS+1)*0.7), EPOCHS], 
                     [LEARNING_RATE/5.0, LEARNING_RATE, LEARNING_RATE/5.0, 0.04])


lr_schedule = lambda t: schedule[int(t)]  #int cast PBO


#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]  #PBO commented

global_step = tf.train.get_or_create_global_step() ## uncommented PBO
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)

##the below is replaced with preprocessing 
def data_aug(x, y):
    return (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)


#### Define a loss function 

In [0]:
def lossfn(model , x , y):
  loss , _ = model(x , y)
  
  return loss
  

In [0]:
#oss_scale_manager = tf.contrib.mixed_precision.FixedLossScaleManager(9000000)

# Wraps the original optimizer in a LossScaleOptimizer.
#opt = tf.contrib.mixed_precision.LossScaleOptimizer(opt, loss_scale_manager)

#opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)  ##PBO


In [0]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

#### Run the model for 3 epochs 

##### lr = 0.4
##### momentum = 0.9
##### batch size 512

In [0]:
t = time.time()
#test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

i = 0
localepochs = EPOCHS
for epoch in range(localepochs):
  
  iterator = dataset.make_one_shot_iterator()
  #image_batch, label_batch = iterator.get_next()
  iterator_test = dataset_test.make_one_shot_iterator()
  train_loss = test_loss = train_acc = test_acc = 0.0
  #train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug, num_parallel_calls=20).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  #train_set = dataset
  tf.keras.backend.set_learning_phase(1)

  for (x, y) in iterator : # tqdm(train_set):
    gvs = tfe.implicit_gradients(lossfn)(model, x, y)
    #print(i)
    i += 1
    if gvs[0] is None:
      raise ValueError("No variables provided.")
    applyGrad = True
    for g, v in gvs : # zip(grads, var):
      if (g is None):
        applyGrad = False
      else :
        g += v * WEIGHT_DECAY * BATCH_SIZE
      
    if (applyGrad):
      opt.apply_gradients( gvs, global_step= global_step ) #list(zip(grads, var)) tf.train.get_or_create_global_step()
    
  tf.keras.backend.set_learning_phase(0)
  
  #if (((epoch + 1 == 1) or (epoch + 1 == localepochs ))):
  
  for (x, y) in iterator_test : #test_set:
    loss, correct = model(x, y)
    #print(loss.numpy())
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)    
    
 #   print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)
      
      
  with open('MODELACCY1.txt' , 'a') as f:
      f.write( str(epoch+1))
      f.write("\t")
      f.write(str(time.time() - t))
      f.write("\t")
      f.write(str(lr_schedule(epoch+1))) 
      f.write("\t")
      f.write( str(test_loss / len_test))
      f.write("\t")
      f.write( str(test_acc / len_test))
      
      f.write("\n")


epoch: 1 lr: 0.24000000000000002 train loss: 0.0 train acc: 0.0 val loss: 46.76940234375 val acc: 0.0957 time: 14.404902458190918
epoch: 2 lr: 0.08 train loss: 0.0 train acc: 0.0 val loss: 0.5401712661743164 val acc: 0.6829 time: 22.11995816230774
epoch: 3 lr: 0.04 train loss: 0.0 train acc: 0.0 val loss: 0.0012118663668632507 val acc: 0.952 time: 29.8717782497406


## Achieved 95.2% accuracy in 3 epoch

####epoch: 3 lr: 0.04 train loss: 0.0 train acc: 0.0 val loss: 0.0012118663668632507 val acc: 0.952 time: 29.8717782497406